In [1]:
import numpy as np
import json
from tensorflow.keras.preprocessing.text import tokenizer_from_json
from tensorflow.keras.preprocessing.sequence import pad_sequences
import pandas as pd
import tensorflow as tf

In [2]:
len_vcb = 366549
seed = 123
max_len = 150

In [3]:
with open('/home/rafael/Documents/Faculdade/ICs/IC Direito/codes/pre-processing/3. Fine-Tuning NER rodando no servidor/Resultados em toda base de SP/utils/tokenizer_100k.json') as f: 
    data = json.load(f) 
    tok = tokenizer_from_json(data)

In [4]:
### definindo metricas customizadas

def recall(y_true, y_pred):
    y_pred = tf.cast(K.argmax(y_pred), tf.float32) # transformando o vetor de (None, 150, n_tags) para (None, 150)
    macro_recall = 0
    y_pred = y_pred[y_true != 0] # retira zero
    y_true = y_true[y_true != 0] # retira zero
    equals = y_pred[K.equal(y_true,y_pred)]  # calcula os verdadeiros positivos, ignorando o zero
    
    for i in range(1,n_tags):
        equals_positives = equals[equals == i]  
        true_positives = K.sum(equals_positives/i) # divide por i, para que os valores se tornem 1

        equals_possibles = y_true[K.equal(y_true,K.constant(i))]/i
        possible_positives = K.sum(equals_possibles)

        recall = true_positives / (possible_positives + K.epsilon())
        macro_recall = macro_recall + recall

    macro_recall = macro_recall/(n_tags-1)  

    return macro_recall

def precision(y_true, y_pred):
    y_pred = tf.cast(K.argmax(y_pred), tf.float32) # trannsformando o vetor de (None, 150, n_tags) para (None, 150)
    macro_precision = 0
    y_pred = y_pred[y_true != 0] # retira zero
    y_true = y_true[y_true != 0] # retira zero
    equals = y_true[K.equal(y_true,y_pred)]  # calcula os verdadeiros positivos, ignorando o zero
    
    for i in range(1,n_tags):
        equals_positives = equals[equals == i]  
        true_positives = K.sum(equals_positives/i) # divide por i, para que os valores se tornem 1

        equals_pred_possibles = y_pred[K.equal(y_pred,K.constant(i))]/i
        predicted_positives = K.sum(equals_pred_possibles)

        precision = true_positives / (predicted_positives + K.epsilon())
        macro_precision = macro_precision + precision

    macro_precision = macro_precision/(n_tags-1)

    return macro_precision

def macro_f1(y_true, y_pred):
    prec = precision(y_true, y_pred)
    rec = recall(y_true, y_pred)
    return 2*((prec*rec)/(prec+rec+K.epsilon()))

In [5]:
custom_metrics = {"recall": recall,"precision": precision, "macro_f1": macro_f1}
model = tf.keras.models.load_model(
    '/home/rafael/Documents/Faculdade/ICs/IC Direito/codes/pre-processing/3. Fine-Tuning NER rodando no servidor/Resultados em toda base de SP/SP_TOTAL_1EPOCH.h5',
    custom_objects=custom_metrics)

In [6]:
piaui_df = pd.read_json('/home/rafael/Documents/Faculdade/ICs/IC Direito/data/samples/sample_piaui_edital.json', lines=True)    

In [7]:
piaui_sample = piaui_df.sample(1, random_state=seed)['conteudo'].to_list()
piaui_sample

['AVISO DE INTIMAÇÃO (4ª Vara de PICOS) Processo nº 0000965-89.2009.8.18.0032 Classe: Ação Penal - Procedimento Ordinário Indiciante: MINISTERIO PUBLICO Advogado(s): Indiciado: MAYCON RAMON LEITE MEDEIROS Advogado(s): ALLAN MANOEL DE CARVALHO(OAB/PIAUÍ Nº 6763) DESPACHO: " Designo nova data para continuação da audiência de instrução e julgamento, o dia 19/02/2018 às 12h30min"']

In [8]:
piaui_sample_tokenized = tok.texts_to_sequences(piaui_sample)
pd.DataFrame(piaui_sample_tokenized[0]).T

,0,1,2,3,4,5,6,7,8,9,...,49,50,51,52,53,54,55,56,57,58
0,11645,22,3556,1810,405,2,1,16,46,144634,...,654,6,169,5,201,479,266,7809,79,28327


In [9]:
piaui_sample_tokenized_padded = pad_sequences(maxlen=max_len,
                                    sequences=piaui_sample_tokenized,
                                    padding="post",
                                    truncating="post")
pd.DataFrame(piaui_sample_tokenized_padded)

,0,1,2,3,4,5,6,7,8,9,...,140,141,142,143,144,145,146,147,148,149
0,11645,22,3556,1810,405,2,1,16,46,144634,...,0,0,0,0,0,0,0,0,0,0


In [10]:
predicted_piaui_sample_tokenized_padded = model(piaui_sample_tokenized_padded)

In [11]:
for i,p in enumerate(predicted_piaui_sample_tokenized_padded[0]):
    if (i < len(piaui_sample_tokenized[0])): 
        print(np.argmax(p),tok.index_word[piaui_sample_tokenized[0][i]])

1 AVISO
1 DE
1 INTIMAÇÃO
1 4ª
1 Vara
1 de
1 UNK
1 Processo
1 nº
1 0000965
1 89
1 2009
1 8
1 18
1 0032
1 Classe
1 Ação
1 Penal
1 Procedimento
1 Ordinário
1 UNK
1 MINISTERIO
1 PUBLICO
1 Advogado
1 s
1 Indiciado
1 MAYCON
1 RAMON
1 LEITE
1 MEDEIROS
1 Advogado
1 s
1 ALLAN
1 MANOEL
1 DE
1 CARVALHO
1 OAB
1 PIAUÍ
1 Nº
1 UNK
1 DESPACHO
1 Designo
1 nova
1 data
1 para
1 continuação
1 da
1 audiência
1 de
1 instrução
1 e
1 julgamento
1 o
1 dia
1 19
1 02
1 2018
1 às
1 12h30min
